# Brute force approach for finding factors of a number

In [113]:
def find_factors(n):
  return [i for i in range(1, n + 1) if n % i == 0]

def is_prime(factors, n):
  return len(factors) == 2 and factors[0] == 1 and factors[1] == n

def run(number):
  factors = find_factors(number)
  prime = is_prime(factors, number)
  print(f"Factors of {number}: {factors}")
  print(f"{number} is prime: {prime}")

run(7919)
run(791)
run(79191919)
# run(791919191) # adding another digit increases run time from 2.5s to 25s

Factors of 7919: [1, 7919]
7919 is prime: True
Factors of 791: [1, 7, 113, 791]
791 is prime: False
Factors of 79191919: [1, 79191919]
79191919 is prime: True


This will slow down drastically before we even get to large numbers

In [114]:
import time

for i in range(1, 10):
  start = time.time()
  run(int('9' * i))
  print(f"-- {time.time() - start} seconds for {i} digits --")

Factors of 9: [1, 3, 9]
9 is prime: False
-- 2.9802322387695312e-05 seconds for 1 digits --
Factors of 99: [1, 3, 9, 11, 33, 99]
99 is prime: False
-- 1.0251998901367188e-05 seconds for 2 digits --
Factors of 999: [1, 3, 9, 27, 37, 111, 333, 999]
999 is prime: False
-- 3.409385681152344e-05 seconds for 3 digits --
Factors of 9999: [1, 3, 9, 11, 33, 99, 101, 303, 909, 1111, 3333, 9999]
9999 is prime: False
-- 0.0003368854522705078 seconds for 4 digits --
Factors of 99999: [1, 3, 9, 41, 123, 271, 369, 813, 2439, 11111, 33333, 99999]
99999 is prime: False
-- 0.0037288665771484375 seconds for 5 digits --
Factors of 999999: [1, 3, 7, 9, 11, 13, 21, 27, 33, 37, 39, 63, 77, 91, 99, 111, 117, 143, 189, 231, 259, 273, 297, 333, 351, 407, 429, 481, 693, 777, 819, 999, 1001, 1221, 1287, 1443, 2079, 2331, 2457, 2849, 3003, 3367, 3663, 3861, 4329, 5291, 6993, 8547, 9009, 10101, 10989, 12987, 15873, 25641, 27027, 30303, 37037, 47619, 76923, 90909, 111111, 142857, 333333, 999999]
999999 is prime: Fal

# gmpy2 to find prime factors
We'll use a library with more efficient algorithm to find factors of larger numbers. There are other algorithms, such as General Number Field Sieve (GNFS) that are significantly more performant but also much more complex.

In [115]:
import gmpy2
import sympy

def find_all_factors(N):
    """
    Use gmpy2's isqrt and trial division to find all factors of N.
    """
    factors = set()
    try:
        # Try small prime trial division using sympy
        for prime in sympy.primerange(2, 10000):
            while N % prime == 0:
                factors.add(prime)
                N //= prime

        # Use isqrt for attempting factorization
        sqrt_n = gmpy2.isqrt(N)
        for x in range(2, sqrt_n + 1):
            while N % x == 0:
                factors.add(x)
                N //= x

        # If N is still greater than 1, it must be a prime factor
        if N > 1:
            factors.add(N)
    except Exception as e:
        print(f"An error occurred while running gmpy2: {e}")

    return sorted(factors)

# efficient up to 10^18
N = 876543210987654321
start = time.time()
factors = find_all_factors(N)
end = time.time()
if factors:
    print(f"All prime factors of {N} are: {factors}")
else:
    print(f"Failed to find any factors of {N}.")
print(f"-- {end - start} seconds for {len(str(N))} digits --")

All prime factors of 876543210987654321 are: [3, 1181, 27489046037183]
-- 0.1816999912261963 seconds for 18 digits --


While this is able to complete 18 digit numbers quite quickly, it will likely time out with 19 digit numbers.

# Pollards Rho to find prime factors
This will give us an increase to be able to factor sightly larger numbers

In [116]:
import gmpy2
import sympy
import random

def pollards_rho(N, seed=2):
    """
    Pollard's Rho algorithm for integer factorization.
    """
    if N % 2 == 0:
        return 2
    x = seed
    y = seed
    d = 1
    f = lambda z: (z * z + 1) % N
    while d == 1:
        x = f(x)
        y = f(f(y))
        d = gmpy2.gcd(abs(x - y), N)
    if d == N:
        return None
    return d

def find_all_factors(N):
    """
    Use gmpy2's isqrt, trial division, and Pollard's Rho to find all factors of N.
    """
    factors = set()
    try:
        # Try small prime trial division using sympy
        for prime in sympy.primerange(2, 10000):
            while N % prime == 0:
                factors.add(prime)
                N //= prime

        # Use Pollard's Rho for larger factors
        while N > 1 and not gmpy2.is_prime(N):
            factor = pollards_rho(N)
            if factor is None:
                break
            while N % factor == 0:
                factors.add(factor)
                N //= factor

        # Use isqrt for attempting factorization
        sqrt_n = gmpy2.isqrt(N)
        for x in range(2, sqrt_n + 1):
            while N % x == 0:
                factors.add(x)
                N //= x

        # If N is still greater than 1, it must be a prime factor
        if N > 1:
            factors.add(N)
    except Exception as e:
        print(f"An error occurred while running gmpy2: {e}")

    return sorted(factors)

N = 20496382304121724099
start = time.time()
factors = find_all_factors(N)
end = time.time()
if factors:
    print(f"All factors of {N} are: {factors}")
else:
    print(f"Failed to find any factors of {N}.")
print(f"-- {end - start} seconds for {len(str(N))} digits --")

All factors of 20496382304121724099 are: [11, 37, mpz(157627), mpz(319486266191)]
-- 0.03428196907043457 seconds for 20 digits --
